In [1]:
import pandas as pd
from pgmpy.models import BayesianModel
from tqdm import tqdm
import numpy as np
from pgmpy.inference import VariableElimination
import random
import networkx as nx
import pylab as plt
from create_network import create_network
import random

In [2]:
train_df = pd.read_csv('train_features_binary.csv')
train_df.drop('Unnamed: 0',inplace=True,axis=1)

In [3]:
corr_df = pd.read_csv('feature_correlation.csv')
corr_df.drop('Unnamed: 0',inplace=True,axis=1)

In [4]:
corr_df

,feature1,feature2,correlation
0,c-52,c-42,0.924619
1,c-73,c-13,0.923344
2,c-26,c-13,0.921875
3,c-33,c-6,0.914730
4,c-55,c-11,0.914637
...,...,...,...
110964,g-560,g-24,0.300016
110965,c-13,g-696,0.300006
110966,g-761,g-533,0.300004
110967,g-711,g-8,0.300004


In [5]:
def make_clusters(n_clusters, max_conns_per_cluster, corr_df):
    clusters = []
    nodes_in_network = []
    for i in range(n_clusters):
        conns = []
        nodes_in_cluster = []
        for corr in range(len(corr_df)):
            if len(nodes_in_cluster) == 0 and corr_df['feature1'][corr] not in nodes_in_network and corr_df['feature2'][corr] not in nodes_in_network:
                conns.append((corr_df['feature1'][corr],corr_df['feature2'][corr]))
                nodes_in_cluster.append(corr_df['feature1'][corr])
                nodes_in_cluster.append(corr_df['feature2'][corr])
                
            if len(nodes_in_cluster) > 0 and (corr_df['feature1'][corr] in nodes_in_cluster or corr_df['feature2'][corr] in nodes_in_cluster) and corr_df['feature1'][corr] not in nodes_in_network and corr_df['feature2'][corr] not in nodes_in_network:
                conns.append((corr_df['feature1'][corr],corr_df['feature2'][corr]))
                nodes_in_cluster.append(corr_df['feature1'][corr])
                nodes_in_cluster.append(corr_df['feature2'][corr])
                conns = list(set(conns))
                if len(conns) == max_conns_per_cluster:
                    clusters = clusters + conns
                    nodes_in_network = nodes_in_network + nodes_in_cluster
                    print(conns)
                    print(f'broke at corr {corr}')
                    break
    clusters = list(set(clusters))
    return clusters

In [6]:
clusters = make_clusters(3,20,corr_df)

[('c-4', 'c-52'), ('c-85', 'c-31'), ('c-52', 'c-42'), ('c-4', 'c-13'), ('c-62', 'c-42'), ('c-4', 'c-42'), ('c-55', 'c-90'), ('c-4', 'c-55'), ('c-52', 'c-31'), ('c-42', 'c-94'), ('c-6', 'c-42'), ('c-60', 'c-94'), ('c-2', 'c-42'), ('c-10', 'c-42'), ('c-2', 'c-13'), ('c-10', 'c-82'), ('c-31', 'c-42'), ('c-63', 'c-42'), ('c-4', 'c-2'), ('c-66', 'c-42'), ('c-42', 'c-82'), ('c-2', 'c-40'), ('c-26', 'c-38'), ('c-2', 'c-94'), ('c-13', 'c-42'), ('c-10', 'c-13'), ('c-55', 'c-38'), ('c-66', 'c-82'), ('c-73', 'c-94'), ('c-51', 'c-42')]
broke at corr 40
[('c-81', 'g-744'), ('g-386', 'g-349'), ('g-37', 'g-72'), ('g-489', 'g-257'), ('g-50', 'g-257'), ('g-744', 'g-72'), ('c-18', 'c-61'), ('g-50', 'g-761'), ('g-72', 'g-63'), ('c-18', 'g-672'), ('g-37', 'g-50'), ('g-50', 'g-672'), ('g-672', 'g-186'), ('g-50', 'g-185'), ('g-37', 'g-63'), ('g-489', 'g-50'), ('g-672', 'g-185'), ('g-121', 'g-672'), ('g-50', 'g-58'), ('g-439', 'g-50'), ('g-37', 'g-369'), ('g-50', 'g-186'), ('g-386', 'g-63'), ('g-37', 'g-761'

In [7]:
clusters

[('c-4', 'c-52'),
 ('c-85', 'c-31'),
 ('g-744', 'g-72'),
 ('c-18', 'c-61'),
 ('g-50', 'g-761'),
 ('c-55', 'c-90'),
 ('c-18', 'g-672'),
 ('g-37', 'g-50'),
 ('c-44', 'c-91'),
 ('c-1', 'c-11'),
 ('g-672', 'g-186'),
 ('c-33', 'c-45'),
 ('c-52', 'c-31'),
 ('c-44', 'c-33'),
 ('c-36', 'c-11'),
 ('c-10', 'c-82'),
 ('c-31', 'c-42'),
 ('c-83', 'c-93'),
 ('g-50', 'g-58'),
 ('g-37', 'g-369'),
 ('c-45', 'c-20'),
 ('c-2', 'c-40'),
 ('g-37', 'g-257'),
 ('c-55', 'c-38'),
 ('c-33', 'c-15'),
 ('g-744', 'g-349'),
 ('c-81', 'g-744'),
 ('c-52', 'c-42'),
 ('g-489', 'g-257'),
 ('c-4', 'c-42'),
 ('g-50', 'g-257'),
 ('g-37', 'g-63'),
 ('g-672', 'g-185'),
 ('c-83', 'c-17'),
 ('g-439', 'g-50'),
 ('c-13', 'c-42'),
 ('c-61', 'g-123'),
 ('c-8', 'c-11'),
 ('c-66', 'c-82'),
 ('c-73', 'c-94'),
 ('c-51', 'c-42'),
 ('c-10', 'c-42'),
 ('c-17', 'c-91'),
 ('g-37', 'g-672'),
 ('g-386', 'g-349'),
 ('g-37', 'g-72'),
 ('c-4', 'c-13'),
 ('c-96', 'c-54'),
 ('g-72', 'g-63'),
 ('c-4', 'c-55'),
 ('c-43', 'c-11'),
 ('g-50', 'g-185')

In [8]:
model, nodes_added = create_network(2,2,clusters)

In [9]:
len(nodes_added)

52

In [10]:
#model.fit automatically finds cpds for each node
model.fit(train_df)

#compute variable eliminations for node independencies to speed up model 
infer = VariableElimination(model)

KeyboardInterrupt: 

In [ ]:
nx.draw(model, with_labels=True)
plt.show()

In [ ]:
model.get_parents('target')